In [ ]:
import torch
from torch import Tensor
import torch.nn as nn
import math

from botorch.models import SingleTaskGP
from botorch.test_functions import Hartmann
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.generation import get_best_candidates, gen_candidates_torch
from botorch.optim import gen_batch_initial_conditions
from botorch.acquisition import qExpectedImprovement, qUpperConfidenceBound
from botorch.sampling import SobolQMCNormalSampler
import os
from typing import Any, Callable, Dict, List, NoReturn, Optional, Tuple, Type, Union
from botorch.test_functions.base import BaseTestProblem
from botorch.acquisition.acquisition import AcquisitionFunction
from botorch.acquisition.cached_cholesky import CachedCholeskyMCSamplerMixin # Updated import
from botorch.acquisition.objective import (
    IdentityMCObjective,
    MCAcquisitionObjective,
    PosteriorTransform,
)
from botorch.acquisition.utils import prune_inferior_points
from botorch.exceptions.errors import UnsupportedError
from botorch.models.model import Model
from botorch.posteriors.posterior import Posterior
from botorch.sampling.base import MCSampler
from botorch.sampling.normal import SobolQMCNormalSampler
from botorch.utils.transforms import (
    concatenate_pending_points,
    match_batch_shape,
    t_batch_mode_transform,
)
from botorch.acquisition.monte_carlo import MCAcquisitionFunction




def clamp_tensor(tensor: torch.Tensor, bounds: torch.Tensor) -> torch.Tensor:
    """
    Clamps the values of a tensor within given bounds.

    Args:
        tensor (Tensor): The tensor to clamp.
        bounds (Tensor): A tensor containing the lower and upper bounds.

    Returns:
        Tensor: The clamped tensor.
    """
    if bounds.dim() == 2 and bounds.size(0) == 2:
        # Common bounds for all samples: bounds shape is (2, dim)
        lower_bounds, upper_bounds = bounds
    elif bounds.dim() == 3 and bounds.size(1) == 2:
        # Per-sample bounds: bounds shape is (batch_size, 2, dim)
        lower_bounds = bounds[:, 0, :]
        upper_bounds = bounds[:, 1, :]
    else:
        raise ValueError("Invalid bounds dimension. Expected bounds of shape (2, dim) or (batch_size, 2, dim).")
    
    # Perform clamping
    return torch.max(torch.min(tensor, upper_bounds), lower_bounds)
